<a href="https://colab.research.google.com/github/kridtapon/WFO-Williams-Aroon-Momentum/blob/main/WFO_Williams_Aroon_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Williams %R
def calculate_williams_r(data, period=14):
    """
    Calculate Williams %R (Williams Percent Range).
    """
    high_max = data['High'].rolling(window=period).max()
    low_min = data['Low'].rolling(window=period).min()
    williams_r = -100 * (high_max - data['Close']) / (high_max - low_min)
    return williams_r

# Function to calculate Aroon Oscillator
def calculate_aroon(data, period=14):
    """
    Calculate Aroon Oscillator.
    """
    aroon_up = (data['High'].rolling(window=period).apply(lambda x: np.argmax(x) + 1) / period) * 100
    aroon_down = (data['Low'].rolling(window=period).apply(lambda x: np.argmin(x) + 1) / period) * 100
    aroon_oscillator = aroon_up - aroon_down
    return aroon_oscillator

# Walk-forward optimization with Williams %R and Aroon Oscillator
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Parameter grids for Williams %R and Aroon Oscillator
    williams_periods = range(5, 31)  # Range for Williams %R periods
    aroon_periods = range(5, 31)  # Range for Aroon Oscillator periods

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through parameter combinations
        for params in itertools.product(williams_periods, aroon_periods):
            williams_period, aroon_period = params

            # Calculate indicators on the training data
            train_data['Williams_R'] = calculate_williams_r(train_data, period=williams_period)
            train_data['Aroon_Oscillator'] = calculate_aroon(train_data, period=aroon_period)

            # Generate entry and exit signals based on Williams %R and Aroon Oscillator
            entries = (train_data['Williams_R'] > -50) & (train_data['Aroon_Oscillator'] > 0)
            exits = (train_data['Williams_R'] < -50) & (train_data['Aroon_Oscillator'] < 0)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = params

        # Test with the best parameters on the test data
        test_data['Williams_R'] = calculate_williams_r(test_data, period=best_params[0])
        test_data['Aroon_Oscillator'] = calculate_aroon(test_data, period=best_params[1])

        entries = (test_data['Williams_R'] > -50) & (test_data['Aroon_Oscillator'] > 0)
        exits = (test_data['Williams_R'] < -50) & (test_data['Aroon_Oscillator'] < 0)

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'SPY'  # Choose your stock symbol
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine entry/exit signals based on the best parameters
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    yearly_data['Williams_R'] = calculate_williams_r(yearly_data, period=params[0])
    yearly_data['Aroon_Oscillator'] = calculate_aroon(yearly_data, period=params[1])

    entries = (yearly_data['Williams_R'] > -50) & (yearly_data['Aroon_Oscillator'] > 0)
    exits = (yearly_data['Williams_R'] < -50) & (yearly_data['Aroon_Oscillator'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-6-0133053f46b4>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0133053f46b4>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0133053f46b4>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in


Walk-Forward Optimization Results:
   Year Best_Params
0  2020     (5, 20)
1  2021    (18, 10)
2  2022     (6, 12)
3  2023    (14, 15)
4  2024     (8, 12)
5  2025    (24, 12)


<ipython-input-6-0133053f46b4>:120: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0133053f46b4>:121: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-0133053f46b4>:120: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-01330

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           138831.671425
Total Return [%]                        38.831671
Benchmark Return [%]                    94.584964
Max Gross Exposure [%]                      100.0
Total Fees Paid                        7382.86698
Max Drawdown [%]                        19.430262
Max Drawdown Duration                       584.0
Total Trades                                   33
Total Closed Trades                            33
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            39.393939
Best Trade [%]                          19.284608
Worst Trade [%]                         -6.324969
Avg Winning Trade [%]                    6.309135
Avg Losing Trade [%]                    -2.230117


In [3]:
# Buy and Hold Performance Metrics
df_holding = df['Close']

# Filter data for the test period (2020-2025)
df_holding = df_holding[(df_holding.index.year >= 2020) & (df_holding.index.year <= 2025)]

pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,863853.052913
Total Return [%],763.853053
Benchmark Return [%],763.853053
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],38.422773
